In [15]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [16]:
df = pd.read_csv(r"C:\Users\caovi\OneDrive\Desktop\projet annuel\core\plank_model\data\plank_dataset_full.csv")
print(df.shape)
df.head()


(3935, 133)


,label,x0,y0,z0,v0,x1,y1,z1,v1,x2,...,z30,v30,x31,y31,z31,v31,x32,y32,z32,v32
0,0,0.830130,0.351851,-0.025262,0.999119,0.850655,0.345155,-0.014917,0.999088,0.851492,...,-0.051480,0.831482,0.031709,0.376000,0.180639,0.640694,0.002836,0.391812,-0.140807,0.875498
1,0,0.827971,0.351691,-0.026928,0.999033,0.848859,0.345107,-0.015630,0.999009,0.849683,...,-0.075619,0.827558,0.033213,0.375629,0.180146,0.626701,0.011874,0.391841,-0.171767,0.869638
2,0,0.826663,0.350897,-0.034896,0.998833,0.848075,0.344515,-0.022585,0.998824,0.848937,...,-0.074106,0.822614,0.033985,0.374774,0.204634,0.610258,0.014278,0.392639,-0.170130,0.862897
3,0,0.825896,0.350415,-0.040929,0.998722,0.847541,0.344141,-0.028329,0.998713,0.848437,...,-0.060950,0.820857,0.037479,0.374753,0.218800,0.599725,0.018048,0.393976,-0.154337,0.859343
4,0,0.825994,0.350126,-0.047208,0.998716,0.847727,0.343751,-0.034991,0.998706,0.848619,...,-0.061091,0.820887,0.038908,0.374778,0.217422,0.595844,0.021588,0.394186,-0.154317,0.858819


In [18]:
X = df.drop(columns=["label"]).values.astype(np.float32)
y = df["label"].values.astype(np.float32)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=True, stratify=y
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_test  = torch.tensor(X_test, dtype=torch.float32)

y_train = torch.tensor(y_train, dtype=torch.float32)
y_test  = torch.tensor(y_test, dtype=torch.float32)

train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)
test_loader  = DataLoader(TensorDataset(X_test, y_test), batch_size=32)


In [19]:
class MLP_Full(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.net(x)

model = MLP_Full(input_dim=X_train.shape[1])


In [20]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [21]:
EPOCHS = 20

for epoch in range(EPOCHS):

    model.train()
    total_loss = 0

    for xb, yb in train_loader:
        preds = model(xb).squeeze()
        loss = loss_fn(preds, yb)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {total_loss/len(train_loader):.4f}")


Epoch 1/20 - Loss: 0.2358
Epoch 2/20 - Loss: 0.0691
Epoch 3/20 - Loss: 0.0340
Epoch 4/20 - Loss: 0.0202
Epoch 5/20 - Loss: 0.0138
Epoch 6/20 - Loss: 0.0075
Epoch 7/20 - Loss: 0.0038
Epoch 8/20 - Loss: 0.0036
Epoch 9/20 - Loss: 0.0046
Epoch 10/20 - Loss: 0.0015
Epoch 11/20 - Loss: 0.0023
Epoch 12/20 - Loss: 0.0008
Epoch 13/20 - Loss: 0.0014
Epoch 14/20 - Loss: 0.0035
Epoch 15/20 - Loss: 0.0027
Epoch 16/20 - Loss: 0.0028
Epoch 17/20 - Loss: 0.0151
Epoch 18/20 - Loss: 0.0069
Epoch 19/20 - Loss: 0.0070
Epoch 20/20 - Loss: 0.0012


In [22]:
model.eval()
with torch.no_grad():
    preds = model(X_test).squeeze()
    preds = torch.sigmoid(preds)
    preds = (preds > 0.5).float()

accuracy = (preds == y_test).float().mean()
print("Accuracy:", accuracy.item())

Accuracy: 1.0


In [ ]:
torch.save(model.state_dict(), r"C:\Users\caovi\OneDrive\Desktop\projet annuel\core\plank_model\model\plank_mlp_full.pt")
print("Modèle FULL sauvegardé.")

Modèle FULL sauvegardé.


: 